# Objetivo 2

In [16]:
!pip install catboost
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt

df = pd.read_csv('')

In [ ]:
######################
# Avaliação do desempenho de modelos criados com dados até determinada semana epidemiológica.
# O conjunto de dados será dividido em fraçoes do banco com dados até determinada semana epidemiológica. O modelo será treinado com esses dados e seu desempenho avaliado. 
# Esse processo se repetirá progressivamente até que todas semanas tenham sido agrupadas.

df_obj2 = pd.DataFrame()
for ano_sem in pd.unique(df.ANO_SEM_NOT.sort_values()):
  df2 = df[df.ANO_SEM_NOT <= ano_sem].copy() # define sub-conjunto de dados até determinada semana
  freq_acum = len(df2) / len(df)
  df2 = df2.drop(columns=('ANO_SEM_NOT'))
  try:
    print('INICIANDO processando da semana ', ano_sem)
    # Divide os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
    df2.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df2['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42 # define a semente de aleatorização
    )
    # Treinamento do modelo
    model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
    model.fit(X_train, y_train)
    # Calculo auc
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    taxa_auc = roc_auc / 0.745
    # Registro do resultado
    df_obj2 = df_obj2.append({'ano_semana': ano_sem, 'N_Registros': len(df2), 'Freq_Acum': freq_acum, 'AUC': roc_auc, 'taxaAUC': taxa_auc}, ignore_index=True)
    print('FIM do processando da semana ', ano_sem)
    print()
  except:
    print('ERRO na semana ', ano_sem)
    df_obj2.append({'ano_semana': ano_sem, 'N_Registros': len(df2), 'Freq_Acum': freq_acum, 'AUC': 0 , 'taxaAUC': taxa_auc}, ignore_index=True)
    print()


In [ ]:
#######
# Construção do gráfico de desempenho dos modelos
#
# Altera tipo de dados da ano_semana
df_obj2['ano_semana'] = df_obj2['ano_semana'].apply(lambda x: str(int(x)))
#
fig, ax1 = plt.subplots(figsize=(20, 10))
#
# Configurando o gráfico de barras para N_Registros
color = 'tab:green'
ax1.bar(df_obj2['ano_semana'], df_obj2['N_Registros'], color='darkgrey', alpha=0.2, label='N_Registros')
ax1.set_xlabel('Semana epidemiológica')
ax1.set_ylabel('Número de Registros', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.tick_params(axis='x', rotation=45)

# Criando um segundo eixo para as linhas
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('%', color='black')

ax2.plot(df_obj2['ano_semana'], df_obj2['AUC'], color='blue', label='AUC')
ax2.tick_params(axis='y', labelcolor='black')

# Adicionando linhas verticais em 0.75 e 1 no eixo ax2
ax2.axhline(y=0.75, color='gray', linestyle='--', linewidth=1)
ax2.axhline(y=0.824661044973545, color='gray', linestyle='--', linewidth=1)

# Adicionando uma linha vertical e uma caixa de texto
ax1.axvline(x='2013', color='gray', linestyle='--', linewidth=1)
ax1.text(0.08, 0.92, 'Semana: 2013\nN Registros: 2.690\nAUC: 0,824661',
         transform=ax1.transAxes, backgroundcolor='white', ha='left', va='center')
ax1.text(0.96, 0.15, 'AUC: 0,75',
         transform=ax1.transAxes, backgroundcolor='white', ha='left', va='center')

# Definindo legenda para todas as linhas e barras
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.title('N Registros e Desempenho por semana epidemiológica')
plt.tight_layout()

plt.show()

In [ ]:
############
# Avaliação do desempenho do primeiro melhor modelo com dados de cada semana epidemiológica
# O modelo será treinado com a fração dos dados relativos ao melhor resultado encontrado e os dados de cada semana epidemiológica processados por esse modelo.


# df para os resultados do processamento
df_obj2_2_Resultado = pd.DataFrame()

######
# Treino do modelo com TODOS os dados até 2013.
df2_2Treino = df[df.ANO_SEM_NOT <= 2013].copy()
df2_2Treino = df2_2Treino.drop(columns=('ANO_SEM_NOT'))
X_df2_2Treino = df2_2Treino.drop(columns=['DESFECHO'])
y_df2_2Treino = df2_2Treino['DESFECHO']
# treinamento do modelos
model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
model.fit(X_df2_2Treino, y_df2_2Treino)

######
# Criação da df futura/teste com dados após 2013.
# O teste é feito com TODOS registros pós 2013, uma vez que o treinamento foi feito com os dados pré 2013
df2_2Teste = df[df.ANO_SEM_NOT > 2013].copy()
#
# loop processando resultado com os dados de cada semana epidemiológica
#
for ano_sem in pd.unique(df2_2Teste.ANO_SEM_NOT.sort_values()):
  df2_2Teste_semana = df2_2Teste[df2_2Teste.ANO_SEM_NOT == ano_sem].copy() # seleciona o registro da semana epidemiológica
  freq_acum = len(df2_2Teste_semana) / len(df)
  df2_2Teste_semana = df2_2Teste_semana.drop(columns=('ANO_SEM_NOT'))
  X_df2_2Teste_semana = df2_2Teste_semana.drop(columns=['DESFECHO']) # Criação do df de variáveis independentes para semana epidemiológica
  y_df2_2Teste_semana = df2_2Teste_semana['DESFECHO'] # Criação do df de variável desfecho para semana epidemiológica
  try:
    print('INICIANDO processando da semana ', ano_sem)
    # Calculo AUC da semana epidemiológica
    y_pred_proba = model.predict_proba(X_df2_2Teste_semana)[:, 1]
    fpr, tpr, _ = roc_curve(y_df2_2Teste_semana, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    # Calculo da acurácia da semana epidemiológica
    y_pred = model.predict(X_df2_2Teste_semana)
    accuracy = accuracy_score(y_df2_2Teste_semana, y_pred)

    df_obj2_2_Resultado = df_obj2_2_Resultado.append({'ano_semana': ano_sem, 'N_Registros': len(df2_2Teste_semana), 'Freq_Acum': freq_acum, 'AUC': roc_auc, 'Acurácia': accuracy}, ignore_index=True)
    print('FIM do processando da semana ', ano_sem)
    print()
  except:
    print('ERRO na semana ', ano_sem)
    df_obj2_2_Resultado = df_obj2_2_Resultado.append({'ano_semana': ano_sem, 'N_Registros': len(df2_2Teste_semana), 'Freq_Acum': freq_acum, 'AUC': 0 }, ignore_index=True)
    print()

In [ ]:
#######
# Construção do gráfico de desempenho do primeiro melhor modelo processando os dados de cada semana epidemiológica. 
# Utiliza os dados processados acima no df_obj2_2_Resultado

# Altera 'ano_semana' para string - importante para formatação do eixo X
df_obj2_2_Resultado['ano_semana'] = df_obj2_2_Resultado['ano_semana'].apply(lambda x: str(int(x)))

######
fig, ax1 = plt.subplots(figsize=(20, 10))

# Configurando o gráfico de barras para N_Registros
color = 'tab:green'
ax1.bar(df_obj2_2_Resultado['ano_semana'], df_obj2_2_Resultado['N_Registros'], color='darkgrey', alpha=0.2, label='N_Registros')
ax1.set_xlabel('Semana epidemiológica')
ax1.set_ylabel('Número de Registros', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.tick_params(axis='x', rotation=45)

# Criando um segundo eixo para as linhas
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('%', color='black')
ax2.plot(df_obj2_2_Resultado['ano_semana'], df_obj2_2_Resultado['AUC'], color='blue', label='AUC')
ax2.plot(df_obj2_2_Resultado['ano_semana'], df_obj2_2_Resultado['Acurácia'], color='green', label='Acurácia')
ax2.tick_params(axis='y', labelcolor='black')

# Adicionando linhas verticais em 0.75 e 1 no eixo ax2
ax2.axhline(y=0.70, color='gray', linestyle='--', linewidth=1)
ax2.axhline(y=0.75, color='gray', linestyle='--', linewidth=1)
ax2.axhline(y=0.6270149710916261, color='red', linestyle='dotted', linewidth=2)

# Adicionando uma linha vertical e uma caixa de texto
ax1.text(0.96, 0.78, 'AUC: 0.75',
         transform=ax1.transAxes, backgroundcolor='white', ha='left', va='center')
ax1.text(0.96, 0.50, 'AUC: 0.70',
         transform=ax1.transAxes, backgroundcolor='white', ha='left', va='center')
ax1.text(0.96, 0.06, 'AUC: 0.62',
         transform=ax1.transAxes, backgroundcolor='white', ha='left', va='center')

# Definindo legenda para todas as linhas e barras
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.title('N Registros e Desempenho para cada semana epidemiológica')
plt.tight_layout()

plt.show()